# Never a sity so real


# translate common script

In [1]:
import env
import utils

# We use Gemini directly instead through langchain
# because of safety settings,
# langchain is developing this and not hasn't released yet
import google.generativeai as genai
import os

genai.configure(
    api_key=os.environ.get("GOOGLE_API_KEY")
)
llm_gemini = genai.GenerativeModel(
    model_name = "gemini-pro",
    safety_settings=utils.safety_config
)


In [13]:
from langchain_community.document_loaders import PyPDFLoader

def translate_pdf(input_file, output_file):
    loader = PyPDFLoader(input_file)
    pages = loader.load_and_split()

    with open(output_file, "w") as output:
        for index, page in enumerate(pages[:]):
            prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
            Your audiences are Chinese native young students in Junior school about 13 years old. 
            Translate the following english text to simplified Chinese: \ ```{page.page_content}``` """ 
    
            response = llm_gemini.generate_content(
                prompt,
                safety_settings=utils.safety_config
            )

            output.write(f"\n---- page: {index} ---- \n")
            if len(response.candidates) > 0:
                # we only get the first candidate
                for p in response.candidates[0].content.parts:
                    output.write(p.text.strip())
                output.flush()

In [ ]:
# markdown
def translate_md(input_file, output_file):
    pass

In [7]:
# TXT
from langchain.text_splitter import CharacterTextSplitter

def translate_txt(input_file, output_file):
    with open(input_file) as f:
        doc_input = f.read()

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    texts = text_splitter.split_text(doc_input)
    print(len(texts))

    # translate
    file_output = open(output_file, "a")

    for index, text in enumerate(texts[:]):
        prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
        Your audiences are Chinese native young students in Junior school about 13 years old. 
        Translate the following english text to simplified Chinese: \ ```{text}``` """ 
    
        response = llm_gemini.generate_content(
            prompt,
            safety_settings=utils.safety_config
        )

        for p in response.parts:
            file_output.write(p.text.strip())

    file_output.close()

In [9]:
# Sroach_econ cdf - 2024/04/07
input_file = f"content/sroach_econ_cdf.txt"
output_file = f"content/sroach_econ_cdf_cn.txt"
translate_txt(input_file, output_file)

2


In [12]:
# tissot-156-en - 2024/04/07
input_file = f"content/tissot-156-en.pdf"
output_file = f"content/tissot-156-en-cn.txt"
translate_pdf(input_file, output_file)
